In [280]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [282]:
import numpy as np
import pandas as pd

In [283]:
from ecephys.data import paths
from ecephys.scoring import load_visbrain_hypnogram, filter_states
from ecephys.signal.sharp_wave_ripples import load_sharp_waves, get_events_in_interval
import ecephys.plot as eplt

In [300]:
SUBJECT = "Segundo"
CONDITIONS = ["REC-6-4", "REC-4-2", "REC-2-0"]
RECORDING_LENGTHS = [7200, 7200, 7200]

In [301]:
ew_hypnograms = [load_visbrain_hypnogram(subject=SUBJECT, condition=condition) for condition in CONDITIONS]

In [302]:
def concatenate_hypnograms(hypnograms, recording_lengths):
    for i in range(1, len(hypnograms)):
        hypnograms[i]['start_time'] = hypnograms[i]['start_time'] + np.cumsum(recording_lengths)[i - 1]
        hypnograms[i]['end_time'] = hypnograms[i]['end_time'] + np.cumsum(recording_lengths)[i - 1]
    
    return pd.concat(hypnograms).reset_index(drop=True)

In [303]:
hypnogram = concatenate_hypnograms(ew_hypnograms, RECORDING_LENGTHS)

In [304]:
ew_spws = [load_sharp_waves(subject=SUBJECT, condition=condition) for condition in CONDITIONS]

In [305]:
def concatenate_sharp_waves(spws_list, recording_lengths):
    for i in range(1, len(spws_list)):
        spws_list[i]['start_time'] = spws_list[i]['start_time'] + np.cumsum(recording_lengths)[i - 1]
        spws_list[i]['end_time'] = spws_list[i]['end_time'] + np.cumsum(recording_lengths)[i - 1]
        spws_list[i]['midpoint'] = spws_list[i]['midpoint'] + np.cumsum(recording_lengths)[i - 1]
    
    spws = pd.concat(spws_list).reset_index(drop=True)
    spws.index += 1
    spws.index = spws.index.rename('spw_number')
    
    return spws

In [306]:
spws = concatenate_sharp_waves(ew_spws, RECORDING_LENGTHS)

In [307]:
def plot_spw_summary(spws, hypnogram, binwidth=10, figsize=(20, 8)):
    fig, (density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(4, 1, figsize=figsize)
    eplt.plot_spw_density(spws, binwidth=binwidth, ax=density_ax)
    spws.plot.scatter(x='start_time', y='duration', ax=duration_ax, title='Event duration (s)', s=1)
    spws.plot.scatter(x='start_time', y='sink_amplitude', ax=amplitude_ax, title='Sink amplitude (mA/mm)', s=1)
    spws.plot.scatter(x='start_time', y='sink_integral', ax=integral_ax, title='Sink integral (mA * ms)', s=1)
    for ax in (density_ax, duration_ax, amplitude_ax, integral_ax):
        ax.set_xlim(density_ax.get_xlim())
        ax.set_xticks(density_ax.get_xticks())
        ax.set_xticklabels(density_ax.get_xticklabels())
        ax.set_xlabel(density_ax.get_xlabel())
        eplt.plot_hypnogram_overlay(hypnogram, ax=ax)
    plt.tight_layout()

In [308]:
plot_spw_summary(spws, hypnogram)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [309]:
def add_spw_properties_to_hypnogram(spws, hypnogram):
    for i, bout in hypnogram.iterrows():
        bout_events = get_events_in_interval(spws, bout.start_time, bout.end_time)
        hypnogram.at[i, 'spw_density'] = len(bout_events) / bout.duration
        hypnogram.at[i, 'mean_duration'] = bout_events.duration.mean()
        hypnogram.at[i, 'mean_sink_amplitude'] = bout_events.sink_amplitude.mean()
        hypnogram.at[i, 'mean_sink_integral'] = bout_events.sink_integral.mean()
    
    hypnogram.fillna(value=0, inplace=True)

In [310]:
add_spw_properties_to_hypnogram(spws, hypnogram)

In [311]:
qwk = filter_states(hypnogram, ['qWk']).reset_index(drop=True)

In [312]:
def plot_spw_summary_by_bout_rank(hypnogram, figsize=(20, 8)):
    fig, (density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(4, 1, figsize=figsize)
    
    hypnogram.plot.scatter(x='index', y='spw_density', ax=density_ax, title='SPW density (Hz)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_duration', ax=duration_ax, title='Event duration (s)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_amplitude', ax=amplitude_ax, title='Sink amplitude (mA/mm)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_integral', ax=integral_ax, title='Sink integral (mA * ms)', s=1)
    for ax in (density_ax, duration_ax, amplitude_ax, integral_ax):
        ax.set_xlabel('Bout rank')
    plt.tight_layout()

In [313]:
plot_spw_summary_by_bout_rank(qwk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [314]:
awk = filter_states(hypnogram, ['aWk']).reset_index(drop=True)

In [315]:
plot_spw_summary_by_bout_rank(awk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …